<a href="https://colab.research.google.com/github/systemis/challenge-2/blob/main/code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [215]:
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
from os import listdir 
from os.path  import isfile, join
from google.colab import files

In [216]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Question 2

In [217]:
data_in = [f for f in listdir('/content/drive/MyDrive/data') if isfile(join('/content/drive/MyDrive/data', f))]

In [218]:
dataf = data_in
studentlist = list()
for name in dataf: 
  name = name.replace('.png', '').split('_')
  if len(name) < 4: 
    print(name)
    continue
  student = [name[0], name[1], name[2], name[3]]
  studentlist.append(student)
studentsdf = pd.DataFrame(studentlist, columns=['Student ID', 'Surname', 'First Name', 'Code'])
studentsdf

['.DS', 'Store']


,Student ID,Surname,First Name,Code
0,2000129,Ho,CongThanh,3A
1,2000104,Truong,ThiDiem,3A
2,2000318,Le,Ngoc,3A
3,20020027,Nguyen,SonTung,3A
4,2000147,Ngo,ThiKhanhLy,3A
...,...,...,...,...
76,2000140,Nguyen,CongHung,3A
77,2000112,Nguyen,TranThienManh,3A
78,2000136,Phan,BoiTran,3A
79,2000114,Le,QuangOach,3A


In [219]:
studentsdf.to_csv('student.csv')
files.download('student.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Question 3

In [220]:
height = 600 
width = 463

In [221]:
img_list = list()
for name in filename: 
  if name == '.DS_Store': continue
  img = plt.imread('/content/drive/MyDrive/data/' + name)
  img = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA)
  img = img[90:, :]
  img_list.append(img)

In [222]:
answerchoice = ['A', 'B', 'C', 'D', 'E']
answer = 5 
part = 5
npart = 6
halfquest = 30
quest = 60

def grid(): 
  _grid = [] 
  for i in range(quest): 
    _grid.append([])
    for j in range(answer): 
      _grid[i].append(0)
  return _grid

In [223]:
def detectAnswerData(): 
  answer_data = ['' for i in range(quest)]
  answer_image = plt.imread('/content/drive/MyDrive/3A.png')
  answer_image = cv2.resize(answer_image, (width, height), interpolation = cv2.INTER_AREA)
  answer_image = answer_image[90:, :]
  checked_data = detect_checked(answer_image)
  for i in range(len(checked_data)): 
    for j in range(len(checked_data[i])): 
      if checked_data[i][j]: 
        answer_data[i] = character_table[j]
  return answer_data

In [224]:
def detect_checked(img, only_firstfive=False): 
  img_copy = img.copy()
  left_side = img[:, 80:200]
  right_side = img[:, 290:410]
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blur_img = cv2.GaussianBlur(gray_img, (5, 5), 0)

  left_side = left_side[:490, :]
  right_side = right_side[:490, :]
  sides = [left_side, right_side]

  data_out = grid()
  if only_firstfive: 
    for o in range(1): 
      part_height = sides[o].shape[0] // npart
      for pc in range(1): 
        check_part(sides, o, pc, data_out)
    return data_out
  for o in range(2): 
    part_height = sides[o].shape[0] // npart
    for pc in range(npart): 
      check_part(sides, o, pc, data_out)
  return data_out

In [225]:
def check_part(sides, pst, pc, data_out): 
  part = sides[pst][(80*pc):80*(pc+1), :]
  part = part[5:70, :]
  gray = cv2.cvtColor(part, cv2.COLOR_BGR2GRAY)
  _, threshold = cv2.threshold(gray, 20, 5, cv2.THRESH_BINARY_INV)

In [226]:
def checkDataInFirstFive(): 
  student_data_copy = studentlist.copy()
  for (i, image) in enumerate(img_list): 
    student = studentlist[i].copy()
    checked_data = detect_checked(image, only_firstfive=True)
    checked_data = checked_data[:5]
    for question_data in checked_data: 
      for j in range(len(question_data)): 
        if question_data[j] == 1: 
          student.append(character_table[j])
    student_data_copy[i] = student

  columns = ['Student ID', 'Surname', 'First Name', 'Code']
  for i in range(5): columns.append('Question: '+str((i+1)))
  data_frame = pd.DataFrame(student_data_copy, columns=columns)
  print('--- Checked of first five question of all student: -----')
  print(data_frame)
  print('\n')